# HW2: Airport Terminal Design

#### Authors: Cameron Irmas, Hannah Mankle

### 1

#### Description: With text and code, show how you check if you have achieved the goal.

In order to determine if we have reached the goal, we need to ensure that:
1. The desired number of terminals has been reached
2. No rules are violated anywhere in the matrix

Assuming that we apply the rules as stated at the transition point (children are valid), we can maintain a number of terminals for any given child node, and check against that, avoiding the need for extra computation (though needs to be accounted for in the transition process).

In [ ]:
using CSV, DataFrames
using DataStructures

In [ ]:
function _read_csv(filename)
    println("Reading CSV...")
    m = CSV.read(filename, DataFrame; header=false)
    return Matrix{Int}(m)
end

In [ ]:
struct AirportNode
    matrix
    depth::Int
    terminals::Int
    g::Int
    h::Int
    f::Int

    function AirportNode(matrix, depth, terminals; g=0, h=0)
        return new(matrix, depth, terminals, g, h, g+h)
    end
end

In [1]:
function _get_neighbors(matrix, (row, col); diagonal=true)
    if diagonal
        neighbors = []
        for nrow=row-1:row+1
            for ncol=col-1:col+1
                if nrow == row && ncol == col
                    continue
                else
                    push!(neighbors, matrix[nrow, ncol])
                end
            end
        end
    else
        top = matrix[row-1, col]
        right = matrix[row, col+1]
        bottom = matrix[row+1, col]
        left = matrix[row, col-1]
        neighbors = [top, right, bottom, left]
    end

    return neighbors
end

_get_neighbors (generic function with 1 method)

#### Rule #1
Rule 1 states that at any given index, a 0 can become a 2 if:
 - Bordering a 1 on side
 - Other 3 sides are 0

In [ ]:
function check_rule_1(node, (row, col))
    state = node.matrix

    if state[row, col] != 0
        return false
    end

    neighbors = _get_neighbors(node.matrix, (row, col); diagonal=false)

    # we're looking for [0, 0, 0, 1]
    return sort(neighbors) == [0, 0, 0, 1]
end

####  Rule #2
Rule 2 states that at any given index, a 0 can become a 1 if:
 - Neighbors existing “1”, even if diagonally
 - Doesn’t cause a violation in Rule 1 (handled in transition function)

In [ ]:
function check_rule_2(node, (row, col))
    if node.matrix[row, col] != 0
        return false
    end

    neighbors = _get_neighbors(node.matrix, (row, col))

    return 1 in neighbors
end

### 2

#### Description: With text and code, show how you represent the candidates, and transitions for this problem.

Candidates are represented above as `AirportNode`. We can maintain a current state of the airport in a matrix, the current depth, and the number of terminals present at this current state. As mentioned, we can carefully maintain a  current number of terminals by incrementing whenever a new terminal is added with that new child node.

Transitions in a typical DFS are represented as a list of functions representing transformations, resulting in child nodes to be added to the stack. In this case, we have two transition functions, one representing 0->1, and the other representing 0->2. Each function will evaluate the current state of the matrix, and any given point.

#### Transition functions
Transitions for adding new walkways

In [ ]:
function transition_walkway(node, (row, col))
    state = node.matrix

    if state[row, col] != 0
        return
    end

    if !check_rule_2(node, (row, col))
        return
    end

    new_m = copy(node.matrix)
    new_m[row, col] = 1
    new_node = AirportNode(
        new_m, node.depth+1, node.terminals; g=node.g+1, h=node.h)

    # check the new_m for rule 1 violations
    # new node cannot be directly next to a 2
    neighbors = _get_neighbors(new_node.matrix, (row, col); diagonal=false)

    if 2 in neighbors
        return
    end

    return new_node
end

Transition function for adding new terminals 

In [ ]:
function transition_terminal(node, (row, col))
    if !check_rule_1(node, (row, col))
        return
    end

    new_m = copy(node.matrix)
    new_m[row, col] = 2
    new_node = AirportNode(
        new_m, node.depth+1, node.terminals+1; g=node.g, h=node.h-1)

    return new_node
end

### 3 
#### Description: Write code for and run DFS for the 6 problems above. Set a max depth to 10. This max may or may not be reached in each problem. Show results. What is the average branching factor?



In [ ]:
function airport_dfs(terminals, seed_file, max_depth)
    s_fns = [
        transition_terminal,
        transition_walkway
    ]
    nodes = Stack{AirportNode}()
    initial_state = _read_csv(seed_file)
    println("Running Airport Terminal Solver...")
    seed = AirportNode(initial_state, 1, 0)
    push!(nodes, seed)
    last_node = seed
    i = 0

    while !isempty(nodes)
        node = pop!(nodes)
        last_node = node
        state = node.matrix

        i += 1
        if i % 100000 == 0
            println("\n\nIterations: $(i). Stack size: $(length(nodes))")
        end

        if node.depth > max_depth
            continue
        end

        _is_goal(node) = node.terminals == terminals

        if _is_goal(node)
            return node
        end

        # Run walkway and terminal transition functions on current matrix
        # and add valid nodes to the stack.
        rows, cols = size(state)
        for row=1:rows
            for col=1:cols
                for fn in s_fns
                    new_node = fn(node, (row, col))    

                    if isnothing(new_node)
                        continue
                    end

                    push!(nodes, new_node)
                end
            end
        end
    end
    println("failed. final state:\n$(last_node.matrix)")
end

### 4 
#### Description: Write a transition function, g(n), such that the number of walkways are minimized and a heuristic, h(n), to achieve the target number of G gates. Does the heuristic meet the four criteria (zero at goal, monotonically decreasing, admissible, same units). Discuss. (it's okay if it doesn't).





### 5
#### Description: Run A* on the 6 problems above. Compare results to 3.

In [ ]:
function airport_a_star(terminals, seed_file, max_depth)
    s_fns = [
        transition_terminal,
        transition_walkway
    ]
    nodes = PriorityQueue{AirportNode, Int}()
    initial_state = _read_csv(seed_file)
    println("Running Airport Terminal Solver...")
    seed = AirportNode(initial_state, 1, 0; h=terminals, g=0)
    enqueue!(nodes, seed, seed.f)
    last_node = seed
    i = 0

    while !isempty(nodes)
        node = dequeue!(nodes)
        last_node = node
        state = node.matrix

        i += 1
        if i % 100000 == 0
            println("\n\nIterations: $(i). Stack size: $(length(nodes)). g: $(node.g), h: $(node.h)")
        end

        if node.depth > max_depth
            continue
        end

        _is_goal(node) = node.terminals == terminals

        if _is_goal(node)
            res = """
                Solution found! Iterations: $(i). g: $(node.g). h: $(node.h). \
                Depth: $(node.depth)
            """
            println(res)
            display(node.matrix)
            return node
        end

        # Run walkway and terminal transition functions on current matrix
        # and add valid nodes to the stack.
        rows, cols = size(state)
        for row=1:rows
            for col=1:cols
                for fn in s_fns
                    new_node = fn(node, (row, col))    

                    if isnothing(new_node)
                        continue
                    end

                    enqueue!(nodes, new_node, new_node.f)
                end
            end
        end
    end
    println("failed. final state:\n$(last_node.matrix)")
end


### 6
#### Description: Based on your results. What are the limits of the size of problem that can be solved. What other approaches could be tried to solve complex problems like this?